In [116]:
import pickle
import numpy as np
import torch
from projection_utils import *

In [117]:
SEGEMENTATION_PATH = "/home/yucheng/storage/data/0801/test_gilman_04/segmentation/0022.pickle"
POINTS_PATH = "/home/yucheng/storage/data/0801/test_gilman_04/points/0022.npy"

In [118]:
points = np.load(POINTS_PATH)
with open(SEGEMENTATION_PATH, 'rb') as handle:
    pred_dict_3D = pickle.load(handle)
    masks = pred_dict_3D["masks"]

In [119]:
masks.shape

(14, 1080, 1920)

In [120]:
array1 = np.array([False, False])
array2 = np.array([True, False])
array3 = np.add(array1, array2)
np.add(array2, array3)

array([ True, False])

In [121]:
arr1 = np.array([1,2,3,4])
arr2 = np.array([5,6,7,8])
np.stack((arr1, arr2))

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [122]:
print(masks[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [123]:
def merge_masks(masks):
    num, height, length = masks.shape
    merged_mask = np.zeros((height, length))
    for mask in masks:
        merged_mask = np.add(merged_mask, mask)
    return merged_mask

In [124]:
mask = merge_masks(masks)
points

array([[ 0.841065, -0.045992,  0.251267,  0.      ],
       [ 0.861126, -0.01443 ,  0.247121,  0.      ],
       [ 0.847056,  0.017447,  0.234163,  0.      ],
       ...,
       [ 0.      , -0.      , -0.      ,  0.      ],
       [ 0.      ,  0.      , -0.      ,  0.      ],
       [ 0.      ,  0.      , -0.      ,  0.      ]])

In [125]:
projected_points = lidar2CameraOurs(points[:, [1, 2, 0]])[:,[1,0]]

/home/yucheng/workspace/WCSNG/point_vis/projection/projection_utils.py:12: RuntimeWarning: invalid value encountered in double_scalars
  x = point[0] / point[2]
/home/yucheng/workspace/WCSNG/point_vis/projection/projection_utils.py:13: RuntimeWarning: invalid value encountered in double_scalars
  y = point[1] / point[2]


In [126]:
print(points.shape)
print(projected_points.shape)
potential_points = []
for i in range(points.shape[0]):
    if np.isnan(points[i]).any() or np.isnan(projected_points[i]).any():
        continue
    x = int(projected_points[i][0])
    y = int(projected_points[i][1])
    if x >= 0 and x < mask.shape[0] and y >= 0 and y < mask.shape[1]:
        if mask[x][y] != 0:
            potential_points.append(points[i])
potential_points = tuple(potential_points)
filtered_points = np.stack(potential_points)

(65536, 4)
(65536, 2)


In [127]:
filtered_points.shape

(1197, 4)